# Lab 2

In [ ]:
# notes
# MAKE SURE INTELLISENSE FOR JUPYTER IS ENABLED IN VS CODE SETTINGS

def print_attributes(obj):
    # Fetch all attributes from the object, filtering out any that start with an underscore
    non_unders = [attr for attr in dir(obj) if not attr.startswith('_')]
    if len(non_unders) == 0:
        print("No attributes found.")
    else:
        print(', '.join(non_unders))
    # Join all attributes into a single string separated by commas
   
# the world is represened by class LatLongPort
from mable.extensions.world_ports import LatLongPort 
print("Shipping network attributes: ")
print_attributes(LatLongPort)

# trasnport oppurtunities, class TimeWindowTrade
from mable.shipping_market import TimeWindowTrade 
print("\nTransport opportunities attributes: ")
print_attributes(TimeWindowTrade)

# vessels are represented by class VesselWithEngine
from mable.extensions.fuel_emissions import VesselWithEngine 
print("\nVessel attributes: ")
print_attributes(VesselWithEngine)

# Trading company - contains inform, preinform and receive
from mable.cargo_bidding import TradingCompany # type: ignore
print("\nTrading company attributes: ")
print_attributes(TradingCompany)

# Simple company - contains inform, preinform and receive
from mable.transport_operation import SimpleCompany 
print("\nSimple company attributes: ")
print_attributes(SimpleCompany)

# bidding class Bid
from mable.transport_operation import Bid 
print("\nBid attributes: ")
print_attributes(Bid)

#schedules
from mable.transportation_scheduling import Schedule
print("\nSchedule attributes: ")
print_attributes(Schedule)

# schedule proposal class
from mable.transport_operation import ScheduleProposal
print("\nSchedule proposal attributes: ")
print_attributes(ScheduleProposal) # is completely empty, unsure why




Exercise 2.5.1 - Trading Company class

In [ ]:
from mable.cargo_bidding import TradingCompany
from mable.examples import environment, fleets
from mable.transport_operation import ScheduleProposal

class MyCompany(TradingCompany):
    def propose_schedules(self, trades):
        schedules = {}
        scheduled_trades = []
        i = 0
        while i < len(trades):
            current_trade = trades[i]
            is_assigned = False
            j = 0
            while j < len(self._fleet) and not is_assigned:
                current_vessel = self._fleet[j]
                current_vessel_schedule = schedules.get(current_vessel, current_vessel.schedule)
                new_schedule = current_vessel_schedule.copy()
                new_schedule.add_transportation(current_trade)
                if new_schedule.verify_schedule():
                    schedules[current_vessel] = new_schedule
                    scheduled_trades.append(current_trade)
                    is_assigned = True
                    j += 1
                    i += 1
        return ScheduleProposal(schedules, scheduled_trades, {})
        
    

    
if __name__ == '__main__':
    specifications_builder = environment.get_specification_builder(environment_files_path=".")
    fleet = fleets.example_fleet_1()
    specifications_builder.add_company(MyCompany.Data(MyCompany, fleet, "My Shipping Corp Ltd."))
    sim = environment.generate_simulation(specifications_builder)
    sim.run()

Exercise 2.5.2 - Improved scheduling

In [ ]:
class MyCompany(TradingCompany):
    def propose_schedules(self, trades):
        schedules = {}
        scheduled_trades = []
        i = 0
        while i < len(trades):
            current_trade = trades[i]
            is_assigned = False
            j = 0
            while j < len(self._fleet) and not is_assigned:
                current_vessel = self._fleet[j]
                current_vessel_schedule = schedules.get(current_vessel, current_vessel.schedule)
                new_schedule = current_vessel_schedule.copy()
                insertion_points = new_schedule.get_insertion_points()
                shortest_schedule = None
                for k in range(len(insertion_points)):
                    idx_pick_up = insertion_points[k]
                    insertion_point_after_idx_k = insertion_points[k:]
                    for m in range(len(insertion_point_after_idx_k)):
                        idx_drop_off = insertion_point_after_idx_k[m]
                        new_schedule_test = new_schedule.copy()
                        new_schedule_test.add_transportation(current_trade, idx_pick_up, idx_drop_off)
                        if (shortest_schedule is None
                        or new_schedule_test.completion_time() < shortest_schedule.completion_time()):
                            if new_schedule_test.verify_schedule():
                                shortest_schedule = new_schedule_test
                if shortest_schedule is not None:
                    schedules[current_vessel] = shortest_schedule
                    scheduled_trades.append(current_trade)
                    is_assigned = True